# Naver movie 개봉 전 review scraping

In [21]:
import re
import math
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver

In [69]:
class NaverMovieScraping:
    
    def __init_(self, url):
        self.url = url
        
    def URLcode(self, url):
        self.url = url
        start = re.search('code=[0-9]+',self.url).group()
        code = re.search('[0-9]+',start).group()
        return code 
    
    def ReviewSCORE(self, code):
        
        self.code = code
        url_base = "http://movie.naver.com/movie/bi/mi/point.nhn?code={}".format(str(code))
        html = urlopen(url_base).read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        graph = soup.findAll('span',{'class':'exp_cnt'})
        star = soup.findAll('div', {'class':'sc_area b_star'})

        exp_idx = [elem.text for elem in graph]
        star_idx = [s.text for s in star]
    
        smile = ''.join(re.findall('[0-9]',exp_idx[0]))
        not_smile= ''.join(re.findall('[0-9]',exp_idx[1]))    
    
        return pd.DataFrame({'보고싶어요' : [smile], 
                         '글쎄요' : [not_smile], 
                         '평점' : [star_idx[0][17:21]], 
                         '참여인원' : [star_idx[0][26:30]]})
    
    def review_page(self, code):
        
        self.code = code
        #url의 경우 따로 개봉전 주소를 넣는다.
        url_base = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=before".format(str(code))
        html = urlopen(url_base).read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        
        score_total = soup.findAll('div', {'class' : 'score_total'})
        review_num = [elem.findChildren('em')[1].get_text() for elem in score_total]
    
        # review_num이 1000개를 넘어가면 , 문자를 제거 
        return math.ceil(int(review_num[0].replace(",","")) / 10)
    
    def getReview(self, code, page_num):
    
        page = int(1)
        #count = int(input('page number : '))
        self.page_num = int(page_num) #review의 전체 페이지
    
        reple_list = []
        score_list = []
        good_list = []
        bad_list = []
        
        self.code = code
        
        while page_num:
        
            #url의 경우 따로 개봉전 주소를 넣는다.
            url_base = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=before&page={}".format(str(code),str(page))
            html = urlopen(url_base).read().decode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
            score_result = soup.find('div', {'class' : 'score_result'})
            li_list = score_result.find_all('li')
        
            for li in li_list:
            
                page = int(page)
                reple = li.find('div', {'class' : 'score_reple'}).find('p').get_text()
            
            #별점이 없는 곳은 0점으로 넣는다.
                try:
                    score = li.find('div', {'class' : 'star_score'}).find('em').get_text()
                except Exception as ex:
                    score = 0
                
                good = li.find('div', {'class' : 'btn_area'}).get_text().split('\n')[1]
                bad = li.find('div', {'class' : 'btn_area'}).get_text().split('\n')[2]
            
                reple_list.append(reple)
                score_list.append(score)
                good_list.append(''.join([s for s in str(good) if s.isdigit()]))
                bad_list.append(''.join([s for s in str(bad) if s.isdigit()]))
            
            page_num -= 1
        
            if not page_num:
                break
            
            page += 1
    
        return pd.DataFrame({'score' : score_list, 'reple' : reple_list, 'good' : good_list, 'bad' : bad_list})


In [70]:
crawler = NaverMovieScraping()

In [71]:
crawler.URLcode('http://movie.naver.com/movie/bi/mi/basic.nhn?code=85579')

'85579'

In [72]:
crawler.ReviewSCORE(85579)

,글쎄요,보고싶어요,참여인원,평점
0,7734,57194,"4,19",8.03


In [73]:
crawler.review_page(85579)

420

In [ ]:
crawler.getReview(85579, 420)